Import des données dans la T_enfants
avec traitement des données multiples a separer avant d'etre integrer inidividuellement 

In [ ]:
import psycopg2
import os
import csv
import pandas as pd
import re

def chemin_fichier()-> str :
    """
        Donne le nom du chemin complet du fichier actuel (.py) 
    """
    chemin=os.getcwd() #methode pour tourner avec jupyter
    #chemin= path.dirname(path.abspath(__file__)) #methode d'Alice avec <os>
    #chemin = str(Path(__file__).parent) #ma methode avec <path>
    #print(chemin)
    return chemin

# etablir une connection
def connexion_DB(fichierDB):
    try:
        conn = psycopg2.connect(user = "santa",
                                password = "claus",
                                host = "localhost",
                                port = "5432:5432",
                                database = fichierDB
                                )

        cur = conn.cursor()
        # Print PostgreSQL Connection properties
        #print ( "Proprietes : ",conn.get_dsn_parameters(),"\n")

        # Print PostgreSQL version
        cur.execute("SELECT version();")
        record = cur.fetchone()
        #print("Version de PostgreSQL : ", record,"\n")
        cur.close()
    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
    
    return conn

def deconnexion_DB(conn):
#closing database connection.
    if(conn):
        conn.close()
        print("PostgreSQL connection is closed")


In [ ]:
conn=connexion_DB("santa_data")
cursor = conn.cursor()

In [ ]:
# transfert des données Enfants dans un DataFrame pour traitement des listes 
if(conn):     
    #suppression des enregistrements de la T_pays avant de refaire tourner le programme
    cursor.execute("DELETE FROM T_enfants")
             
    rq_select='''
            SELECT id_l , nom_enfant ,age, id_p
            FROM T_complete
            ORDER BY id_l
            ;
            '''     
    insert_data_query = f""" 
                        INSERT INTO T_enfants
                        (ID_e,
                        ID_l,
                        Nom_enfant,
                        Age,
                        ID_p) 
                        VALUES (default,%s,%s,%s,%s)
                        """
    
    cursor.execute(rq_select)
    res=cursor.fetchall()
       
    count=0
    for row in res :
        l_enfant=re.split(' et ?| ou | & |/|,', row[1])   
        nb_enfant=len(l_enfant)
        nb_age=0
        if row[2] is not None : #age
            l_age=re.split('\s*,\s*|\s*et\s*|\s*an.?|\s*an ?|\s*', row[2])
            nb_age=len(l_age)
            
        if nb_enfant > 1 :
            for i in range(0,nb_enfant) :
                l_insert=[row[0]] #id_l
                l_insert.append(l_enfant[i]) #nom_enfant
                if nb_age>2 :
                    l_insert.append(float(l_age[i]))
                    
                elif nb_age == 2 :
                    l_insert.append(float(l_age[0]))
                    
                elif row[2] is None :
                        l_insert.append(row[2]) # age
            
                l_insert.append(row[3]) #id_p
                t_insert=tuple(l_insert)
                print('M',t_insert)
                
                #insertion dans T_enfant
                cursor.execute(insert_data_query,t_insert)
                conn.commit
                count += cursor.rowcount
                            
        elif nb_enfant==1 :
            l_insert=[row[0],row[1]] #id_l   nom_enfant
            if row[2] is None :
                l_insert.append(row[2]) # age
                
            else :
                l_insert.append(float(l_age[0]))
                
            l_insert.append(row[3]) #id_p
            t_insert=tuple(l_insert)
            print('u',t_insert)
            
            #insertion dans T_enfant
            cursor.execute(insert_data_query,t_insert)
            conn.commit()
            count += cursor.rowcount
                
        
            
    print (count, "Record inserted successfully into T_enfants")
    

In [ ]:
rq_select='''
          SELECT 
              ID_e,
              ID_l,
              nom_enfant,
              age,
              TE.ID_p, 
              Pays,ID_l 
          FROM T_enfants TE LEFT JOIN T_pays TP
          ON TE.ID_p=TP.ID_p
          ORDER BY id_l;
          '''

cursor.execute(rq_select)
res=cursor.fetchall()

print('\nID_e'.ljust(8),'ID_l'.ljust(7),'nom_enfant'.ljust(21),'age'.ljust(15),'ID_p'.ljust(8),'Pays'.ljust(20))
trait="-"*80
print(trait)
for row in res:
    print(str(row[0]).ljust(8),end='')
    print(str(row[1]).ljust(8),end='')
    print(str(row[2]).ljust(25)[0:22],end='')
    print(str(row[3]).ljust(15),end='')
    print(str(row[4]).ljust(8),end='')
    print(str(row[5]).ljust(15))

In [ ]:
cursor.close()
deconnexion_DB(conn)